<a href="https://colab.research.google.com/github/jayasrijerri/FMML-lab/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?Yes, averaging validation accuracy across multiple splits (e.g., using **k-fold cross-validation**) generally gives more consistent and reliable results compared to a single train-test split. Here's why and how it works:

---

### **Why Averaging Across Splits is Effective**
1. **Mitigates Variability in Data Splits:**
   - The performance of a model on a single train-test split can vary significantly depending on how the data is divided. For example, a split with an overrepresentation of certain classes or outliers can skew the results.
   - By averaging across multiple splits, you smooth out these fluctuations, leading to a more robust estimate of model performance.

2. **Utilizes the Entire Dataset:**
   - In **k-fold cross-validation**, every data point is used for both training and validation at least once (except in the case of leave-one-out cross-validation, where it's used multiple times). This ensures the model is evaluated on diverse subsets of the data, giving a better indication of its generalization ability.

3. **Reduces Overfitting Risk in Evaluation:**
   - Using multiple splits ensures that the validation metric is less dependent on any single subset of data, which can sometimes lead to overly optimistic or pessimistic performance estimates.

4. **Improves Consistency for Small Datasets:**
   - When the dataset is small, a single split might not represent the true data distribution. Cross-validation compensates for this by providing multiple perspectives on the dataset.

---

### **How Averaging Works in Cross-Validation**
1. **Perform k-Fold Cross-Validation:**
   - Divide the dataset into \(k\) folds (e.g., 5 or 10).
   - Train the model on \(k-1\) folds and validate it on the remaining fold.
   - Repeat this process \(k\) times, each time using a different fold for validation.

2. **Compute Validation Accuracy for Each Fold:**
   - After training on \(k-1\) folds, compute the accuracy on the held-out fold.

3. **Average the Results:**
   - Take the mean of the \(k\) validation accuracies to get the final estimate:
     \[
     \text{Mean Accuracy} = \frac{1}{k} \sum_{i=1}^{k} \text{Accuracy}_i
     \]
   - Optionally, you can also calculate the **standard deviation** to understand the variability of the results.

---

### **When to Use Averaging**
- **Small Datasets:** Cross-validation is essential to make the most of limited data.
- **Model Comparison:** Averaging helps reduce biases when comparing different models or hyperparameter configurations.
- **Highly Variable Data:** If the dataset has high variance or imbalanced classes, averaging ensures a fair evaluation.

---

### **Potential Downsides**
1. **Computational Cost:**
   - Training the model \(k\) times (or more, in the case of nested cross-validation) can be computationally expensive, especially for large datasets or complex models.

2. **Overestimation with Overlap:**
   - If the train and validation sets overlap (e.g., in leave-one-out cross-validation with highly similar samples), the validation scores might be overly optimistic.

3. **Choice of \(k\):**
   - A smaller \(k\) (e.g., 5) is computationally lighter but less robust. A larger \(k\) (e.g., 10) provides more accurate estimates but requires more time.

---

### **Conclusion**
Averaging validation accuracy across multiple splits provides more consistent and reliable results than relying on a single split. It captures the model's performance across a broader range of data scenarios and is particularly valuable for small or imbalanced datasets. However, balance the added computational cost with the need for robust evaluation. For large datasets, even a simpler train-validation-test split might suffice, provided it reflects the true data distribution.
2. Does it give more accurate estimate of test accuracy?Yes, averaging validation accuracy across multiple splits (e.g., through **k-fold cross-validation**) generally provides a **more accurate estimate of test accuracy** compared to a single train-test split. Here's why this is the case:

---

### **Why Cross-Validation Provides a Better Estimate of Test Accuracy**

1. **Reduces Bias from a Single Split**:
   - A single train-test split might not represent the entire dataset's distribution, especially if the split is not random or if the dataset is small or imbalanced.
   - Cross-validation ensures that every data point is used for both training and validation, reducing the bias introduced by any one split.

2. **Accounts for Data Variability**:
   - Real-world datasets often have inherent variability. Some splits might inadvertently include more difficult examples in the validation set or unbalanced representations of classes.
   - By averaging across multiple splits, cross-validation captures the performance on a variety of train-test scenarios, making the estimate more robust.

3. **Improved Generalization Assessment**:
   - Since each fold tests the model on unseen data (the held-out fold), cross-validation simulates the process of making predictions on a separate test set multiple times. This gives a better sense of how the model is likely to generalize to entirely new data.

4. **Mitigates Overfitting in Evaluation**:
   - If the model overfits to the training data in a single train-test split, the test accuracy might appear artificially high or low. Averaging over multiple splits smooths out these effects and provides a more reliable measure.

---

### **When Cross-Validation Gives an Accurate Estimate of Test Accuracy**

1. **When the Dataset is Representative**:
   - The dataset must reflect the characteristics of the unseen test data. Cross-validation estimates will only be accurate if the dataset is not biased or unrepresentative.

2. **When \(k\) is Chosen Appropriately**:
   - A smaller \(k\) (e.g., 5) works well for large datasets, while a larger \(k\) (e.g., 10) is better for small datasets, as it ensures more samples are included in training.

3. **When There's No Data Leakage**:
   - If features or information from the validation folds inadvertently leak into the training folds, cross-validation accuracy can be misleading.

---

### **Caveats**
While cross-validation provides a better estimate of test accuracy, it is not perfect. Consider the following:

1. **Data Distribution in Test Set**:
   - If the test set distribution is significantly different from the training set (e.g., due to domain shifts), even a good cross-validation accuracy will not accurately predict test performance.

2. **Overfitting to Validation Folds**:
   - If cross-validation is used extensively for hyperparameter tuning, there's a risk of overfitting to the cross-validation procedure, leading to optimistic estimates.

3. **Computational Cost**:
   - Cross-validation can be computationally expensive, especially for large datasets or complex models.

---

### **Practical Tip: Use a Hold-Out Test Set**
To ensure the cross-validation estimate aligns with the true test accuracy:
1. Perform \(k\)-fold cross-validation on the training data to evaluate model performance and tune hyperparameters.
2. After finalizing the model, evaluate it on a completely unseen **test set** that was held out from the start.

This two-step process ensures:
- Cross-validation provides a reliable estimate during model development.
- The final test set gives an unbiased measure of real-world performance.

---

### **Summary**
Averaging validation accuracy across multiple splits gives a more accurate estimate of test accuracy because it reduces biases from individual splits, captures data variability, and better assesses generalization. However, complement it with a hold-out test set for a final unbiased evaluation.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?The number of iterations in a validation process, such as cross-validation, directly impacts the reliability of the estimate of test accuracy. Here’s how increasing iterations affects the estimate:

---

### **Effects of Higher Iterations**

1. **Reduced Variance in the Estimate**:
   - With a higher number of iterations (e.g., increasing \(k\) in \(k\)-fold cross-validation or running repeated cross-validation), the estimate becomes more stable and consistent.
   - Each iteration provides a different view of the data, averaging out peculiarities caused by random splits.
   - As the number of iterations increases, the variance in validation results decreases, leading to a more reliable estimate.

2. **Improved Robustness**:
   - More iterations ensure the model is tested on various subsets of the data. This captures more diverse data characteristics, especially important if the dataset is small or has a lot of variability.
   - It reduces the influence of any particular split that may be overly favorable or unfavorable.

3. **Law of Large Numbers**:
   - As the number of iterations increases, the average accuracy converges toward the true performance metric, giving a better approximation of the test accuracy.

4. **Diminishing Returns**:
   - After a certain point, increasing the number of iterations yields marginal improvements. For example:
     - A 5-fold cross-validation already uses all data points for training and validation once. Repeated 5-fold cross-validation (e.g., 10 repetitions) improves the estimate but may not dramatically change it beyond a certain number of repetitions.
     - Computational costs increase linearly with the number of iterations, but the improvement in the accuracy estimate may taper off.

---

### **When More Iterations Improve the Estimate**

1. **Small or Imbalanced Datasets**:
   - With fewer samples or skewed class distributions, the variability of each split is higher. A higher number of iterations mitigates this effect.

2. **High Data Variability**:
   - If the data has outliers or noisy patterns, more iterations ensure that the estimate isn't dominated by a few "lucky" or "unlucky" splits.

3. **Algorithm Sensitivity**:
   - Some algorithms, like neural networks or decision trees, are highly sensitive to data variations. More iterations can provide a more balanced evaluation.

---

### **Trade-offs of Higher Iterations**

1. **Computational Cost**:
   - Increasing the number of iterations directly increases the time and resources required for training and validation. This can be significant for large datasets or computationally expensive models.

2. **Overfitting to Validation Procedure**:
   - If repeated iterations are used excessively for hyperparameter tuning, the model might overfit to the cross-validation setup, leading to overly optimistic estimates.

3. **Diminishing Returns**:
   - Beyond a certain number of iterations, the marginal benefit decreases. For example:
     - Moving from 5 to 10 repetitions can make a noticeable difference.
     - Moving from 50 to 100 repetitions provides little improvement but significantly increases computational cost.

---

### **Practical Recommendations**

1. **Choose Reasonable \(k\) for Cross-Validation**:
   - For large datasets, 5- or 10-fold cross-validation is often sufficient.
   - For small datasets, \(k = 10\) or leave-one-out cross-validation (LOOCV) can improve the estimate.

2. **Repeated Cross-Validation**:
   - Perform repeated cross-validation (e.g., 5-fold repeated 3–5 times) to further reduce variance and get a more stable estimate.

3. **Monitor Computational Cost**:
   - Balance the number of iterations with available computational resources. Adding too many iterations may not justify the minor improvement in reliability.

---

### **Summary**
Higher iterations generally yield a better estimate of test accuracy by reducing variance and improving robustness. However, beyond a certain point, the improvements diminish, and computational costs increase. A balanced approach, such as 10-fold cross-validation or repeated 5-fold cross-validation, is often sufficient for most practical tasks.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Yes, increasing the number of iterations (e.g., using repeated cross-validation or leave-one-out cross-validation) can help deal with very small training or validation datasets, but it has limitations and considerations. Here’s how and why it works, along with its challenges:

---

### **How Increasing Iterations Helps with Small Datasets**

1. **Better Utilization of Limited Data**:
   - In a small dataset, every data point is valuable. Techniques like \(k\)-fold cross-validation ensure that each sample is used for both training and validation multiple times, maximizing the utility of the available data.
   - Example: In **leave-one-out cross-validation (LOOCV)**, every sample serves as a validation point once, and the rest are used for training. This ensures the most comprehensive use of data.

2. **Reduced Bias and Variance in the Estimate**:
   - Small datasets are prone to high variability in performance estimates due to randomness in splitting.
   - By increasing the number of iterations, you average over many splits, reducing the bias and variance of the validation accuracy.

3. **Improved Generalization Assessment**:
   - A larger number of splits ensures the model is exposed to different subsets of the data, which improves the robustness of the performance estimate.

---

### **Challenges of Relying on Increased Iterations**

1. **Overfitting to Small Datasets**:
   - Even with many iterations, if the training dataset is very small, the model may overfit to the limited data. This results in overly optimistic validation accuracy that does not generalize to unseen test data.

2. **Lack of Diversity in Small Datasets**:
   - If the dataset is small, simply reusing the same data in different configurations cannot create additional diversity. The model might still fail to generalize well to new data outside the training set.

3. **Computational Cost**:
   - Increasing iterations (e.g., using LOOCV) can be computationally expensive, especially for complex models, as it requires training the model as many times as there are data points or folds.

4. **Validation Performance Might Not Represent True Test Performance**:
   - If the small dataset is not representative of the test data distribution, increasing iterations does not solve the fundamental problem of data insufficiency.

---

### **Strategies to Complement Higher Iterations for Small Datasets**

1. **Data Augmentation**:
   - If applicable, artificially increase the size of the dataset through data augmentation techniques (e.g., transformations for image data, adding noise to text or numerical data).

2. **Use Regularization**:
   - Apply techniques like L1/L2 regularization, dropout (for neural networks), or early stopping to prevent overfitting.

3. **Pretrained Models or Transfer Learning**:
   - Leverage models pretrained on larger datasets and fine-tune them on the small dataset. This helps mitigate the limitations of small data by utilizing prior knowledge.

4. **Simplify the Model**:
   - Use models with fewer parameters to reduce the risk of overfitting. For example, linear models or small decision trees might perform better than deep neural networks on small datasets.

5. **Bootstrap Aggregation (Bagging)**:
   - Use bootstrap sampling (random sampling with replacement) to create multiple training datasets, then average predictions across models trained on these samples.

6. **Combine Cross-Validation with External Test Set**:
   - Use cross-validation for model tuning but still reserve a small portion of the dataset as an independent test set to evaluate final performance.

---

### **When Increasing Iterations Alone Is Effective**
- **Tiny Validation Set**:
   - If you can afford to train on nearly the entire dataset (e.g., LOOCV), high-iteration cross-validation provides a reliable performance estimate for small datasets.

- **Algorithm with Low Overfitting Risk**:
   - Simpler algorithms (e.g., logistic regression, naive Bayes) are less prone to overfitting, making repeated iterations more useful.

- **Few Features Relative to Data Points**:
   - When the dataset is small but not extremely sparse in terms of features, higher iterations can still yield meaningful estimates.

---

### **Key Takeaway**
Increasing iterations helps deal with small training or validation datasets by maximizing the use of available data and reducing variability in estimates. However, it cannot compensate for fundamental limitations of small datasets, such as overfitting risk, lack of diversity, or unrepresentative data. Complement this approach with techniques like data augmentation, transfer learning, and simpler models for best results.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.